# Test Notebook for various TIMIT utilities

*read_seg_file()* is a segmentation reading routine included with a lot of other TIMIT related definitions   

TIMIT alphabets:   TIMIT48, TIMIT41, TIMIT61
TIMIT mappings:    timit61_48, timit48_41, timit61_41

07/03/2022: tested with v0.6

In [ ]:
# optional install of the pyspch package
#!pip install git+https://github.com/compi1234/pyspch.git

In [1]:
%matplotlib inline
import os,sys,io 
import pkg_resources
import scipy.signal

from urllib.request import urlopen
from IPython.display import display, Audio, HTML, clear_output
from ipywidgets import interact

import math,time
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec 

import librosa as librosa
    
import pyspch.sp as Sps
import pyspch.display as Spd
import pyspch.core as Spch

### Get TIMIT alphabets and conversions with builtin tools

In [2]:
timit41= Spch.get_timit_alphabet("timit41")
timit61_41 = Spch.get_timit_mapping("timit61","timit48")
timit41, timit61_41

(['aa',
  'ae',
  'ah',
  'ao',
  'aw',
  'ah',
  'ah',
  'er',
  'ay',
  'b',
  'cl',
  'ch',
  'd',
  'cl',
  'dh',
  't',
  'eh',
  'l',
  'm',
  'n',
  'ng',
  'sil',
  'er',
  'ey',
  'f',
  'g',
  'cl',
  'sil',
  'hh',
  'hh',
  'ih',
  'ih',
  'iy',
  'jh',
  'k',
  'cl',
  'l',
  'm',
  'n',
  'ng',
  'n',
  'ow',
  'oy',
  'p',
  'sil',
  'cl',
  'sil',
  'r',
  's',
  'sh',
  't',
  'cl',
  'th',
  'uh',
  'uw',
  'uw',
  'v',
  'w',
  'y',
  'z',
  'zh'],
 {'aa': 'aa',
  'ae': 'ae',
  'ah': 'ah',
  'ao': 'ao',
  'aw': 'aw',
  'ax': 'ax',
  'ax-h': 'ax',
  'axr': 'er',
  'ay': 'ay',
  'b': 'b',
  'bcl': 'vcl',
  'ch': 'ch',
  'd': 'd',
  'dcl': 'vcl',
  'dh': 'dh',
  'dx': 'dx',
  'eh': 'eh',
  'el': 'el',
  'em': 'm',
  'en': 'en',
  'eng': 'ng',
  'epi': 'epi',
  'er': 'er',
  'ey': 'ey',
  'f': 'f',
  'g': 'g',
  'gcl': 'vcl',
  'h#': 'sil',
  'hh': 'hh',
  'hv': 'hh',
  'ih': 'ih',
  'ix': 'ix',
  'iy': 'iy',
  'jh': 'jh',
  'k': 'k',
  'kcl': 'cl',
  'l': 'l',
  'm': 'm

In [3]:
### Predefined symbol sets , e.g. TIMIT61, TIMIT48, TIMIT41  and mappings only dealing with the differences are also directly available
Spch.TIMIT41, Spch.timit61_41_diff

(['aa',
  'ae',
  'ah',
  'ao',
  'aw',
  'er',
  'ay',
  'b',
  'ch',
  'd',
  'dh',
  'eh',
  'm',
  'ng',
  'ey',
  'f',
  'g',
  'hh',
  'ih',
  'iy',
  'jh',
  'k',
  'l',
  'n',
  'ow',
  'oy',
  'p',
  'r',
  's',
  'sh',
  't',
  'th',
  'uh',
  'uw',
  'v',
  'w',
  'y',
  'z',
  'zh',
  'sil',
  'cl'],
 {'axr': 'er',
  'em': 'm',
  'eng': 'ng',
  'nx': 'n',
  'hv': 'hh',
  'ux': 'uw',
  'kcl': 'cl',
  'pcl': 'cl',
  'tcl': 'cl',
  'h#': 'sil',
  'pau': 'sil',
  'q': 'sil',
  'bcl': 'cl',
  'dcl': 'cl',
  'gcl': 'cl',
  'epi': 'sil',
  'dx': 't',
  'ax-h': 'ah',
  'ix': 'ih',
  'ax': 'ah',
  'el': 'l',
  'en': 'n'})

### Read data from the 'data' directory included in the package

In [4]:
my_file = pkg_resources.resource_filename('pyspch', 'data/friendly.wrd')
print(my_file)
with open(my_file) as f:
    lines = f.readlines()
lines

c:\users\compi\nextcloud\github\pyspch\pyspch\data\friendly.wrd


['.01 .44 friendly\n', '.49 1.04 computers\n']

## TIMIT Alphabets, Mappings, label2indx, ..

we use following naming conventions

- phone:  name of an element in a phonetic alphabet
- idx:    index of a phone in the alphabet
- map_phn2idx, map_idx2phn:  dictionaries for converting between phone and idx
- seg:    segmentation dataframe
- lbls:   (list) alignment sequence of phone labels
- indx:   (list) alignment sequence of phone indices

In [5]:
# unique phone <--> index mappings
phn_set = np.array(Spch.get_timit_alphabet("timit41"))
map_idx2phn = {i : phn for i, phn in enumerate(phn_set) }
map_phn2idx = {phn : i for i, phn in enumerate(phn_set) }
map_idx2phn

{0: 'aa',
 1: 'ae',
 2: 'ah',
 3: 'ao',
 4: 'aw',
 5: 'ah',
 6: 'ah',
 7: 'er',
 8: 'ay',
 9: 'b',
 10: 'cl',
 11: 'ch',
 12: 'd',
 13: 'cl',
 14: 'dh',
 15: 't',
 16: 'eh',
 17: 'l',
 18: 'm',
 19: 'n',
 20: 'ng',
 21: 'sil',
 22: 'er',
 23: 'ey',
 24: 'f',
 25: 'g',
 26: 'cl',
 27: 'sil',
 28: 'hh',
 29: 'hh',
 30: 'ih',
 31: 'ih',
 32: 'iy',
 33: 'jh',
 34: 'k',
 35: 'cl',
 36: 'l',
 37: 'm',
 38: 'n',
 39: 'ng',
 40: 'n',
 41: 'ow',
 42: 'oy',
 43: 'p',
 44: 'sil',
 45: 'cl',
 46: 'sil',
 47: 'r',
 48: 's',
 49: 'sh',
 50: 't',
 51: 'cl',
 52: 'th',
 53: 'uh',
 54: 'uw',
 55: 'uw',
 56: 'v',
 57: 'w',
 58: 'y',
 59: 'z',
 60: 'zh'}

In [6]:
# convert labels to indices usable for slicing
def lbls2indx(lbls,map_phn2idx=None):
    return np.array([ map_phn2idx[phn] for phn in lbls ])
# convert indices to labels for readability
def indx2lbls(indx,map_idx2phn=None):
    return np.array([ map_idx2phn[i] for i in indx ])

In [7]:
indx = [0,3,38,2,2,2,1]
lbls = indx2lbls(indx,map_idx2phn=map_idx2phn)
lbls

array(['aa', 'ao', 'n', 'ah', 'ah', 'ah', 'ae'], dtype='<U2')

In [8]:
indxx = lbls2indx(lbls,map_phn2idx=map_phn2idx)
indxx

array([ 0,  3, 40,  6,  6,  6,  1])

## Segmentations and Alignments

- segmentation:  contains asynchronous segmentation times and labels
- alignment:     a frame synchronous mapping from index to label

the core package provides conversions  seg2lbls() and lbls2seg()
remark that
    - segmentations can have time gaps with unacounted label information
    - alignments must always be complete

In [9]:
help(Spch.seg2lbls)

Help on function seg2lbls in module pyspch.core.utils:

seg2lbls(seg, shift=0.01, n_frames=None, end_time=None, pad_lbl=None)
    seg2lbls() converts a segmentation DataFrame to an alignment 
                in the form of an array of labels
                This conversion will fill empty regions with the padding label
                'pad_lbl' and will make an output at least as large as n_frames (or end_time)
    
    Arguments:
    seg      Segmentation DataFrame with columns {'t0','t1','seg'}
    shift    frame shift (in units used in seg)
    n_frames desired number of frames, overrides last value in seg if larger
    end_time desired end time of segmentation, overrides last value in seg if larger
    pad_lbl  a padding label for missing segments (default=None)
    
    Returns:
    lbls[]   list with labels



In [11]:
# read a datafile
dir='https://homes.esat.kuleuven.be/~spchlab/data/'
file = "timit/si1027"
#file = "timit/test/dr1/faks0/si2203" 
sr=16000.
segphn = Spch.read_seg_file(dir+file+ ".phn",dt=1/sr,fmt='float32',xlat='timit61_41')
segphn

,t0,t1,seg
0,0.000000,0.598750,sil
1,0.598750,0.615000,dh
2,0.615000,0.665750,ih
3,0.665750,0.749812,r
4,0.749812,0.915000,iy
5,0.915000,1.021250,z
6,1.021250,1.046937,ah
7,1.046937,1.196250,n
8,1.196250,1.315000,z
9,1.315000,1.387500,f


In [12]:
pd.DataFrame(Spch.seg2lbls(segphn))

,0
0,sil
1,sil
2,sil
3,sil
4,sil
...,...
346,sil
347,sil
348,sil
349,sil


### Reading TIMIT transcriptions files

In [ ]:
fn1 = 'timit/sa1.txt'
#fn1="timit/phones-61-48-39-41.txt"
fname='https://homes.esat.kuleuven.be/~spchlab/data/'
transcript = Spch.read_data_file(fname+fn1)[0].strip().split(None,2)
print('Samples: ',int(transcript[0]),int(transcript[1]))
print('Transcript: ',transcript[2].strip('.,!?:;'))

## (TIMIT) Segmentation Files

Segmentation Files are assumed to be in the format

t0  t1   seg    
....


#### time units
t0, t1 are begin and end-times of segment 'seg'
the units of time, can be specified in the read_seg_file() module with 'dt', by default segmentation times are given in seconds;
in timit it is often in samples (with SR=16000), thus use dt=1/16000

#### phonetic symbols
Phonetic transcriptions come in a variety of phonetic symbol sets.
These utilities include default definitions (and orderings) of **TIMIT48** and **TIMIT41**.   TIMIT48 is the default used in experiments with TIMIT.   TIMIT41 is our own more compact version inspired by the alphabet in the CMU dictionaries, with 1 additional closure ('cl') symbol.

A number of mappings between the different alphabets are foreseen.  To apply them use the field 'xlat' at time of reading and specify the desired translation:  timit61_48, timit61_41, ..
These are simple dictionary based mappings.   

In [ ]:
# read a datafile
dir='https://homes.esat.kuleuven.be/~spchlab/data/'
file = "timit/si1027" #@param {type:"string"}
wavfile = dir+file+".wav" 
wavdata, sr = Spch.load(wavfile)
spg1 = Sps.spectrogram(wavdata,sample_rate=sr,n_mels=None)

# get segmentations
segwrd = Spch.read_seg_file(dir+file+ ".wrd",dt=1/sr,fmt='float32')
segphn61 = Spch.read_seg_file(dir+file+ ".phn",dt=1/sr,fmt='float32')
segphn = Spch.read_seg_file(dir+file+ ".phn",dt=1/sr,fmt='float32',xlat='timit61_41')

In [ ]:
fig = Spd.PlotSpg(spg1,wavdata=wavdata,segwav=segwrd,segspg=segphn,sample_rate=sr)
display(fig)
display(Audio(data=wavdata,rate=sr))